In [1]:
from transformers import AutoConfig
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from vicuna_llava import vicuna_llava, dataset_llava
from accelerate import Accelerator
accelerator = Accelerator()

Log in to huggingface (using llama requires you to request access on huggingface)

In [2]:

from huggingface_hub import login
login()

### Instantiate vicunallava

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# grab vicuna and its tokenizer
model_name = "meta-llama/Llama-3.2-1B"#"lmsys/vicuna-7b-v1.5"
config = AutoConfig.from_pretrained(model_name)

vicunallava = accelerator.prepare(vicuna_llava(config, llmURL=model_name, accelerator=accelerator))


Grab pretrained projector

In [4]:
#linear_llava_proj = torch.load('mm_projector.bin', weights_only=True)
#linear_llava_weights = linear_llava_proj['model.mm_projector.weight']
#linear_llava_biases = linear_llava_proj['model.mm_projector.bias']

#with torch.no_grad():
#    vicunallava.im_embedding.weight.copy_(linear_llava_weights)
#    vicunallava.im_embedding.bias.copy_(linear_llava_biases)

### Instantiate Dataset

In [6]:
# directories pointing to images directory and chat.json, I've downloaded the dataset locally to deal with some of the missing images
chat = 'CC3M/chat.json'
im_dir = "CC3M/images/"

num_samples=12
subset_idcs = list(range(num_samples))

cc3m_dataset = accelerator.prepare(Subset(dataset_llava(chat, im_dir), subset_idcs))
batch_size=3
cc3m_dataloader =  accelerator.prepare(DataLoader(cc3m_dataset, batch_size=batch_size))

Test Generating on Some Sample from Dataset

In [7]:
#testprompt, testimage, _ = cc3m_dataset[0]
#testimage = testimage.to(device)
#transform = T.ToPILImage()

#output = vicunallava.generate(testimage, testprompt, max_new_tokens=15)
#print(f'input prompt: {testprompt}')
#transform(testimage).show()

#print(f'response: {output}')

### Training (Stage 1)

Set which parameters to be optimized

In [8]:
# don't compute gradient on any vicunallava layers besides im_embedding
for i in iter(vicunallava.parameters()):
    i.requires_grad = False
for i in iter(vicunallava.model.parameters()):
    i.requires_grad = False
for i in iter(vicunallava.im_embedding.parameters()):
    i.requires_grad = True

In [ ]:
from torch.optim import Adam
from transformers import get_scheduler


optimizer =  accelerator.prepare(Adam([{"params":vicunallava.parameters(),
                                        "params":vicunallava.im_embedding.parameters()}], lr=2e-3))

num_epochs = 1000
num_dataset_samples = len(cc3m_dataset)
num_batch_steps = num_epochs*num_dataset_samples/batch_size
num_warmup_steps = np.ceil(0.03*num_batch_steps)
lr_scheduler = get_scheduler("cosine",
                             optimizer=optimizer,
                             num_warmup_steps=num_warmup_steps,
                             num_training_steps=num_batch_steps)

losses = []

vicunallava.train()
for i in range(num_epochs):
    batchiter = 0

    for batchprompt,batchimage,batchresp in cc3m_dataloader:
        
        input = [batchprompt[j]+'###'+batchresp[j] for j in range(batch_size)]
        input = vicunallava.process_prompt(input)
        tokenized_input = vicunallava.tokenize(input)
        outs = vicunallava(batchimage, input, batch_size=batch_size)
        losses.append(outs['loss'])

        # optimize model
        accelerator.backward(outs['loss'])
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        batchiter+=1
    print(f"epoch:{i}, loss:{outs['loss']}")    
    if i%10 == 0:
        torch.save(vicunallava.im_embedding, 'vicunallava_im_embedding_stage1.pt')



c:\Users\colli\OneDrive\Documents\Classes\UofM\ECE 598 (LLMs)\Final Project\llava_finalproject\llava_finalproject\vicuna_llava.py:100: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


epoch:0, loss:11.723671913146973
epoch:1, loss:11.667545318603516
epoch:2, loss:11.67780590057373
epoch:3, loss:11.65052604675293
